# Predicting phases for experimental dataset
# For the large filter dataset

In [75]:
# Import packages
%matplotlib qt
import tensorflow as tf
import numpy as np
import os, glob
import gc
import tqdm
import hyperspy.api as hs
from tempfile import TemporaryFile
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## Import examples of experimental data to process
If only one file to predict, make it a list `data` of a single path.

In [76]:
root = r'C:\Users\Sauron\PycharmProjects\ml_pyxem\mini_2'
folder = r'experimental_data\npz_files'

file_extension = '*K.npz' # Select the q cropped file (rebinned)
#file_extension = '*PX.npz' # Select the px cropped file (no rebin)

path = os.path.join(root,folder,file_extension)
paths = glob.glob(path)
paths.sort()
paths

['C:\\Users\\Sauron\\PycharmProjects\\ml_pyxem\\mini_2\\experimental_data\\npz_files\\20200209_163154_centre_rebin_correct_rb_radial_norm_cropK.npz',
 'C:\\Users\\Sauron\\PycharmProjects\\ml_pyxem\\mini_2\\experimental_data\\npz_files\\roi_3_rebin_radial_norm_cropK.npz',
 'C:\\Users\\Sauron\\PycharmProjects\\ml_pyxem\\mini_2\\experimental_data\\npz_files\\roi_4_rebin_radial_norm_cropK.npz']

## Load the models to predict from
Will search for the model in the same folder or one folder up.

In [77]:
model_file_extension = '*.h5'

# Select which model to use
try:
    model_path = glob.glob(model_file_extension, recursive=True)[0]
except IndexError:
    model_path = glob.glob(os.path.join('..', model_file_extension), recursive=True)[0]

model_path

'..\\20220405_CNN_large_filter_cnn_3nclasses_2epochs_64batchsize__train_22500n_0p9732ac__test_7500n_0.9691ac.h5'

In [78]:
m = tf.keras.models.load_model(model_path)
m.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 142, 64)           448       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 71, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            24640     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 33, 64)           0         
 1D)                                                             
                                                                 
 dropout (Dropout)           (None, 33, 64)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 28, 64)            2

In [80]:
######### Run the predicting

# Load model
model_basename = os.path.basename(model_path).split('.')[0]
model_dir = os.path.dirname(model_path)

for i, fname in enumerate(tqdm.tqdm(paths)):
    exp_name = os.path.basename(fname).split('.')[0]
    exp_npzfile = np.load(fname)
    exp_data = exp_npzfile['exp1d']
    shape = (exp_data.shape[0] * exp_data.shape[1], exp_data.shape[2], 1)
    exp_data_reshape = np.reshape(exp_data, shape)
    exp_preds = m.predict(exp_data_reshape)
    n_phases = exp_preds.shape[-1]
    shape = (exp_data.shape[0], exp_data.shape[1], n_phases)
    exp_pred_reshape = np.reshape(exp_preds, shape)
    # Transpose so phase is the navigation axis
    exp_pred_reshape = np.moveaxis(exp_pred_reshape, -1, 0)
    s = hs.signals.Signal2D(exp_pred_reshape)

    del exp_npzfile
    del exp_data
    del exp_data_reshape
    del exp_preds
    del exp_pred_reshape
    gc.collect()

    # Add phases in the metadata
    metadata_path = glob.glob(os.path.join(model_dir, 'phase_names*.npy'))[0]
    phases = np.load(metadata_path)
    phases = [s for s in phases]
    s.metadata.General.set_item("Phases", phases)
    s.metadata.General.set_item("Model trained", model_basename)

    ## Save results in 2 folders
    # Save the stack with probabilites
    file_name = f'probability_preditions_{exp_name}.hspy'
    s.save(file_name, overwrite=True)

    # Plot prob distribution maps
    f = plt.figure(figsize= (5 * len(phases), 5))
    hs.plot.plot_images(s, vmax=1, vmin=0, cmap='viridis', colorbar='single',
                        label=phases, axes_decor='off', scalebar=[0],
                        suptitle='Probabilty predictions', fig=f, tight_layout=True,)
    plt.savefig(file_name.replace('hspy', 'png'))
    plt.close()

    # Save the sparse categorical results [1 to n]
    signal_cat = hs.signals.Signal2D(s.data.argmax(axis=0))
    signal_cat.metadata.General.set_item("Phases", phases)
    signal_cat.metadata.General.set_item("Model trained", model_basename)
    file_name = f'sparse_categorical_{exp_name}.hspy'
    signal_cat.save(file_name, overwrite=True)

    # Plot categorical file
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    ax = plt.subplot()
    im = ax.imshow(signal_cat.data, cmap='viridis')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax, ticks=np.arange(len(phases)))
    cbar.ax.set_yticklabels(phases)
    plt.title('Argmax value')
    plt.tight_layout()
    plt.savefig(file_name.replace('hspy', 'png'))
    plt.close()

    # Save the one-hot encoded categorical results [0,1,...,0]
    def cont_to_categorical(continous_ar):
        b =  np.zeros_like(continous_ar)
        b[np.argmax(continous_ar)] = 1
        return b

    signal_cat_expanded = s.T.map(cont_to_categorical, inplace=False,)
    signal_cat_expanded.metadata.General.set_item("Phases", phases)
    signal_cat_expanded.metadata.General.set_item("Model trained", model_basename)
    file_name = f'onehot_categorical_{exp_name}.hspy'
    signal_cat_expanded.save(file_name, overwrite=True)


100%|██████████| 3/3 [00:10<00:00,  3.61s/it]


  0%|          | 0/16129 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/728 [00:00<?, ?it/s]